In [ ]:
gameFolder = "DOA++-MAME"
#gameFolder = "SFIII-MAME"
#gameFolder = "UMK3-MAME"
#gameFolder = "TEKTAG-MAME"

In [ ]:
import sys, os
import time
timeDepSeed = int((time.time()-int(time.time()-0.5))*1000)

sys.path.append(os.path.join(os.path.abspath(''), '../../games',gameFolder))   

In [ ]:
from makeDiambraEnv import *

In [ ]:
diambraKwargs = {}
diambraKwargs["roms_path"] = "../../roms/MAMEToolkit/roms/"
diambraKwargs["binary_path"] = "../../../customMAME/"
diambraKwargs["player"] = "P1"
diambraKwargs["frame_ratio"] = 3
#diambraKwargs["render"] =True
#diambraKwargs["throttle"] = False
#diambraKwargs["sound"] = False 
#diambraKwargs["character"] ="Random"
diambraKwargs["character"] ="Kasumi"

numEnv=1

wrapperKwargs = {}
wrapperKwargs["frame_stack"] = 4
wrapperKwargs["clip_rewards"] = False
wrapperKwargs["normalize_rewards"] = True
wrapperKwargs["scale"] = True
wrapperKwargs["hw_obs_resize"] = [84, 84]

keyToAdd = None
#keyToAdd.append("action")
#keyToAdd.append("player")
#keyToAdd.append("healthP1")
#keyToAdd.append("healthP2")
#keyToAdd.append("positionP1")
#keyToAdd.append("positionP2")
#keyToAdd.append("winsP1")
#keyToAdd.append("winsP2")

env = make_diambra_env(diambraMame, env_prefix="Test", num_env=numEnv, seed=timeDepSeed,
                       diambra_kwargs = diambraKwargs, wrapper_kwargs = wrapperKwargs,
                       key_to_add = keyToAdd, no_vec = True)

# OR 
#env = make_diambra_env(diambraMame, num_env=numEnv, seed=0, diambra_kwargs = diambraKwargs)
# Frame-stacking with 4 frames
#env = VecFrameStack(env, n_stack=4)

In [ ]:
observation = env.reset()

cumulativeEpRew = 0.0
cumulativeEpRewAll = []
cumulativeTotRew = 0.0

maxNumEp = 100
currNumEp = 0

while currNumEp < maxNumEp:

    action = int(input("Action to take"))
    #action = env.action_space.sample()
    print("Action:", action)
    observation, reward, done, info = env.step(action)

    #print("Frames shape:", observation.shape)
    print("Reward:", reward)
    #print("Fighting = ", info["fighting"])
    #print("Rewards = ", info["rewards"])
    #print("HealthP1 = ", info["healthP1"])
    #print("HealthP2 = ", info["healthP2"])
    #print("HealthP1_1 = ", info["healthP1_1"])
    #print("HealthP1_2 = ", info["healthP1_2"])
    #print("HealthP2_1 = ", info["healthP2_1"])
    #print("HealthP2_2 = ", info["healthP2_2"])
    #print("PositionP1 = ", info["positionP1"])
    #print("PositionP2 = ", info["positionP2"])
    #print("WinP1 = ", info["winsP1"])
    #print("WinP2 = ", info["winsP2"])

    
    cumulativeEpRew += reward
    
    if np.any(done):
        currNumEp += 1
        print("Ep. # = ", currNumEp)
        print("Ep. Cumulative Rew # = ", cumulativeEpRew)
        sys.stdout.flush()
        cumulativeEpRewAll.append(cumulativeEpRew)
        cumulativeTotRew += cumulativeEpRew
        cumulativeEpRew = 0.0

        observation = env.reset()

print("Mean cumulative reward = ", cumulativeTotRew/maxNumEp)    
print("Mean cumulative reward = ", np.mean(cumulativeEpRewAll))    
print("Std cumulative reward = ", np.std(cumulativeEpRewAll))       
    
env.close()